# 1. Importações

In [1]:
# ========================== Importações ========================================================================================
import pandas as pd
import numpy as np
import requests
import time
import os
from datetime import datetime
from unidecode import unidecode  # Remove acentos

# 2. Funções auxiliares

In [2]:
# ========================== Funções Auxiliares =================================================================================
def fazer_requisicao_basica(url):
    """Faz a requisição à API e retorna os dados em JSON"""
    try:
        resposta = requests.get(url)
        resposta.raise_for_status()  # Levanta um erro para status 4xx e 5xx
        return resposta.json().get('dados', [])
    except requests.exceptions.RequestException as e:
        print(f"Erro ao acessar {url}: {e}")
        return []
# ===============================================================================================================================
def obter_deputadas():
    """Obtém a lista de deputadas na Câmara"""
    start_time = time.time()  # Início da contagem do tempo
    url = 'https://dadosabertos.camara.leg.br/api/v2/deputados?dataInicio=2020-01-01&dataFim=2024-12-31&ordem=ASC&ordenarPor=nome'
    dados = fazer_requisicao_basica(url)
    end_time = time.time()  # Fim da contagem do tempo
    print(f"Tempo total de execução: {end_time - start_time:.2f} segundos")
    return pd.DataFrame(dados) if dados else pd.DataFrame()
# ===============================================================================================================================
def obter_detalhes_deputadas(df_deputadas):
    """Obtém detalhes individuais das deputadas e retorna um DataFrame"""
    start_time = time.time()
    detalhes = []

    for id_dep in df_deputadas['id']:
        url = f"https://dadosabertos.camara.leg.br/api/v2/deputados/{id_dep}"
        data = fazer_requisicao_basica(url)  # Função para obter os dados da API

        if not data:
            print(f"[ERRO] Nenhum dado retornado para id {id_dep}")
            continue

        # Converter os dados principais da deputada para DataFrame
        df_dados_deputada = pd.json_normalize(data)

        detalhes.append(df_dados_deputada)

    # Consolidar todos os detalhes em um único DataFrame
    df_resultado = pd.concat(detalhes, ignore_index=True) if detalhes else pd.DataFrame()

    elapsed_time = time.time() - start_time
    print(f"Tempo total de execução: {elapsed_time:.2f} segundos")

    return df_resultado
# ===============================================================================================================================
def obter_partidos(df_deputadas_completo):
    """Obtém detalhes dos partidos a partir da sigla e retorna um DataFrame"""
    start_time = time.time()  # Início da contagem do tempo
    url_base = "https://dadosabertos.camara.leg.br/api/v2/partidos?sigla={}&ordem=ASC&ordenarPor=sigla"
    siglas = df_deputadas_completo["siglaPartido"].dropna().unique()  # Pega siglas únicas, ignorando NaN
    dados_partidos = []

    for sigla in siglas:
        sigla_corrigida = unidecode(sigla).upper()  # Remove acentos e coloca em maiúsculas
        url = url_base.format(sigla_corrigida)  
        response = requests.get(url)
        
        if response.status_code == 200:
            data = response.json()
            if "dados" in data and len(data["dados"]) > 0:
                df_partido = pd.json_normalize(data["dados"])
                dados_partidos.append(df_partido)

    # Junta todos os dados coletados em um único DataFrame
    df_partidos = pd.concat(dados_partidos, ignore_index=True) if dados_partidos else pd.DataFrame()

    end_time = time.time()  # Fim da contagem do tempo
    print(f"Tempo total de execução: {end_time - start_time:.2f} segundos")

    return df_partidos
# ===============================================================================================================================
def obter_detalhes_partidos(df_partidos):
    """
    Obtém os detalhes de cada partido utilizando o ID da lista de partidos.

    Parâmetros:
        df_partidos (pd.DataFrame): DataFrame contendo a lista de partidos, incluindo a coluna 'id'.

    Retorna:
        pd.DataFrame: DataFrame com os detalhes dos partidos, incluindo campos expandidos do 'status'.
    """
    start_time = time.time()  # Início da contagem do tempo
    detalhes_partidos = []  # Lista para armazenar os detalhes

    for partido_id in df_partidos["id"]:
        url = f"https://dadosabertos.camara.leg.br/api/v2/partidos/{partido_id}"
        dados = fazer_requisicao_basica(url)

        if dados:
            # Converte os dados principais do partido para um DataFrame
            df_partido = pd.json_normalize(dados)

            # Expande os campos dentro de "status", se existir
            if "status" in dados:
                df_status = pd.json_normalize(dados["status"])
                df_detalhado = pd.concat([df_partido, df_status], axis=1)  # Junta os DataFrames
            else:
                df_detalhado = df_partido  # Caso não haja status, mantém apenas os dados básicos

            detalhes_partidos.append(df_detalhado)

    end_time = time.time()  # Fim da contagem do tempo
    print(f"Tempo total de execução: {end_time - start_time:.2f} segundos")

    # Concatena todos os DataFrames na lista em um único DataFrame
    return pd.concat(detalhes_partidos, ignore_index=True) if detalhes_partidos else pd.DataFrame()
# ===============================================================================================================================
def determinar_geracao(ano_nascimento):
    if ano_nascimento >= 1997:
        return "Geração Z (1997-2010)"
    elif 1981 <= ano_nascimento <= 1996:
        return "Millennials - Geração Y (1981-1996)"
    elif 1965 <= ano_nascimento <= 1980:
        return "Geração X (1965-1980)"
    elif 1946 <= ano_nascimento <= 1964:
        return "Baby Boomers (1946-1964)"
    else:
        return "Geração Silenciosa (até 1945)"
# ===============================================================================================================================
def obter_lideres_legislaturas():
    """Obtém os líderes das legislaturas 56 e 57 e retorna um DataFrame"""
    start_time = time.time()
    legislaturas = [56, 57]
    lideres = []

    for legislatura in legislaturas:
        url = f"https://dadosabertos.camara.leg.br/api/v2/legislaturas/{legislatura}/lideres"
        dados = fazer_requisicao_basica(url)

        if not dados:
            print(f"[ERRO] Nenhum dado retornado para legislatura {legislatura}")
            continue

        df_dados_lideres = pd.json_normalize(dados)
        df_dados_lideres['legislatura'] = legislatura  # Adiciona a legislatura para referência

        lideres.append(df_dados_lideres)

    elapsed_time = time.time() - start_time
    print(f"Tempo total de execução: {elapsed_time:.2f} segundos")

    return pd.concat(lideres, ignore_index=True) if lideres else pd.DataFrame()
# ===============================================================================================================================
def obter_sexo_por_id(lista_ids):
    """Consulta a API da Câmara para obter o sexo de parlamentares por ID"""
    dados = []

    for id_ in lista_ids:
        url = f"https://dadosabertos.camara.leg.br/api/v2/deputados/{id_}"
        response = requests.get(url)

        if response.status_code == 200:
            info = response.json()
            sexo = info['dados'].get('sexo', 'Não informado')
            dados.append({'id': id_, 'sexo': sexo})
        else:
            print(f"[ERRO] Não foi possível acessar dados para o ID {id_}")
            dados.append({'id': id_, 'sexo': 'Erro'})

    return pd.DataFrame(dados)

# 3. Extração de dados

In [3]:
# ========================== Tabela Deputadas =================================================================================
print("Baixando dados das deputadas...")
df_deputadas = obter_deputadas()

if not df_deputadas.empty:
    print("Baixando detalhes das deputadas...")
    df_deputadas_detalhes = obter_detalhes_deputadas(df_deputadas)
else:
    df_deputadas_detalhes = pd.DataFrame()

# Chamando a função e fazendo a junção das tabelas deputadas
df_deputadas_completo = df_deputadas.merge(df_deputadas_detalhes, on="id", how="left")

# ========================== Tabela Partidos ===================================================================================
print("Baixando dados dos partidos...")
df_partidos = obter_partidos(df_deputadas_completo)

if not df_partidos.empty:
    print("Baixando detalhes dos partidos...")
    df_detalhes_partidos = obter_detalhes_partidos(df_partidos)
else:
    df_detalhes_partidos = pd.DataFrame()

# Chamando a função e fazendo a junção das tabelas partidos
df_partidos_completo = df_partidos.merge(df_detalhes_partidos, on="id", how="left")

# ========================== Tabela Líderes ====================================================================================
print("Baixando dados dos líderes por legislatura...")
df_lideres_leg = obter_lideres_legislaturas()

Baixando dados das deputadas...
Tempo total de execução: 9.85 segundos
Baixando detalhes das deputadas...
Tempo total de execução: 146.05 segundos
Baixando dados dos partidos...
Tempo total de execução: 118.22 segundos
Baixando detalhes dos partidos...
Tempo total de execução: 6.15 segundos
Baixando dados dos líderes por legislatura...
Tempo total de execução: 0.41 segundos


# 4. Consolidação e Limpeza dos Dados

## 4.1. Tabela Deputadas

In [4]:
# Verificando colunas com dados nulos

df_deputadas_completo.isnull().sum()

id                                   0
uri_x                                0
nome                                 0
siglaPartido                         0
uriPartido                           0
siglaUf                              0
idLegislatura                        0
urlFoto                              0
email                             1610
uri_y                                0
nomeCivil                            0
cpf                                  0
sexo                                 0
urlWebsite                        2262
redeSocial                           0
dataNascimento                       0
dataFalecimento                   2274
ufNascimento                         1
municipioNascimento                  0
escolaridade                        33
ultimoStatus.id                      0
ultimoStatus.uri                     0
ultimoStatus.nome                    0
ultimoStatus.siglaPartido            0
ultimoStatus.uriPartido           2278
ultimoStatus.siglaUf     

In [5]:
# Localizando dados nulos na coluna ufNascimento e preenchendo com os valores da coluna siglaUf

df_deputadas_completo.loc[(df_deputadas_completo['ufNascimento'].isna()), 'ufNascimento'] = df_deputadas_completo['siglaUf']

In [6]:
# Localizando dados nulos na coluna escolaridade e preenchendo com o valor 'não informado'

df_deputadas_completo.loc[(df_deputadas_completo['escolaridade'].isna()), 'escolaridade'] = "Não informado"

In [7]:
# Localizando dados nulos na coluna ultimoStatus.data e preenchendo com a data de hoje

df_deputadas_completo.loc[(df_deputadas_completo['ultimoStatus.data'].isna()), 'ultimoStatus.data'] = datetime.now()

In [8]:
# Verificando a quantidade de registros duplicados em que o id, idLegislatura e siglaPartido são iguais

df_deputadas_completo.duplicated(subset=['id', 'idLegislatura', 'siglaPartido']).sum()

1294

In [9]:
# Excluindo os registros duplicados, com base nos critérios acima, e mantendo apenas o último registro

df_deputadas_completo = df_deputadas_completo.drop_duplicates(subset=["id", "idLegislatura", 'siglaPartido'], keep='last')

In [10]:
# Conferindo se os registros duplicados foram apagados

df_deputadas_completo.duplicated(subset=['id', 'idLegislatura', 'siglaPartido']).sum()

0

In [11]:
# Criando as colunas com as datas de início e fim da legislatura 

df_deputadas_completo["inicioLegislatura"] = np.where(
    df_deputadas_completo["idLegislatura"] == 57, "2023-02-01",
    np.where(df_deputadas_completo["idLegislatura"] == 56, "2019-02-01", np.nan)
)

df_deputadas_completo["fimLegislatura"] = np.where(
    df_deputadas_completo["idLegislatura"] == 57, "2027-01-31",
    np.where(df_deputadas_completo["idLegislatura"] == 56, "2023-01-31", np.nan)
)

In [12]:
# Formatando campos com datas para o formato brasileiro

df_deputadas_completo['dataNascimento'] = pd.to_datetime(df_deputadas_completo['dataNascimento']).dt.strftime('%d/%m/%Y')
df_deputadas_completo['inicioLegislatura'] = pd.to_datetime(df_deputadas_completo['inicioLegislatura']).dt.strftime('%d/%m/%Y')
df_deputadas_completo['fimLegislatura'] = pd.to_datetime(df_deputadas_completo['fimLegislatura']).dt.strftime('%d/%m/%Y')

In [13]:
# Criando o campo falecida (bool)

df_deputadas_completo['falecida'] = df_deputadas_completo['dataFalecimento'].notna()

In [14]:
# Criando o campo Ano de Nascimento

df_deputadas_completo.loc[:, 'ano_nascimento'] = pd.to_datetime(df_deputadas_completo['dataNascimento'], dayfirst=True).dt.year

In [15]:
# Criando o campo Geração a partir da chamada da função Determinar Geração

df_deputadas_completo.loc[:, 'geração'] = df_deputadas_completo['ano_nascimento'].apply(determinar_geracao)

In [16]:
# Criando o campo Idade com base na data de nascimento x data de hoje

df_deputadas_completo.loc[:, 'idade'] = datetime.now().year - df_deputadas_completo['ano_nascimento']

In [17]:
# Excluindo colunas desnecessárias

df_deputadas_completo.drop(columns=['uri_x', 'uriPartido', 'urlFoto', 'email', 'uri_y', 'urlWebsite', 'redeSocial', 
                                    'ultimoStatus.uriPartido', 'ultimoStatus.email', 'ultimoStatus.gabinete.nome', 
                                    'ultimoStatus.gabinete.predio', 'ultimoStatus.gabinete.sala', 'ultimoStatus.gabinete.andar',
                                    'ultimoStatus.gabinete.telefone', 'ultimoStatus.gabinete.email', 'ultimoStatus.descricaoStatus', 
                                    'dataFalecimento'], inplace=True)

In [18]:
# Conferência final

df_deputadas_completo.isnull().sum()

id                                0
nome                              0
siglaPartido                      0
siglaUf                           0
idLegislatura                     0
nomeCivil                         0
cpf                               0
sexo                              0
dataNascimento                    0
ufNascimento                      0
municipioNascimento               0
escolaridade                      0
ultimoStatus.id                   0
ultimoStatus.uri                  0
ultimoStatus.nome                 0
ultimoStatus.siglaPartido         0
ultimoStatus.siglaUf              0
ultimoStatus.idLegislatura        0
ultimoStatus.urlFoto              0
ultimoStatus.data                 0
ultimoStatus.nomeEleitoral        0
ultimoStatus.situacao             0
ultimoStatus.condicaoEleitoral    0
inicioLegislatura                 0
fimLegislatura                    0
falecida                          0
ano_nascimento                    0
geração                     

## 4.2. Tabela Partidos 

In [19]:
# Verifica os campos da tabela em nulo

df_partidos_completo.isnull().sum()

id                             0
sigla_x                        0
nome_x                         0
uri_x                          0
sigla_y                        0
nome_y                         0
uri_y                          0
numeroEleitoral               31
urlLogo                        0
urlWebSite                    31
urlFacebook                   31
status.data                    1
status.idLegislatura           0
status.situacao                1
status.totalPosse              0
status.totalMembros            0
status.uriMembros              0
status.lider.uri               0
status.lider.nome              1
status.lider.siglaPartido      1
status.lider.uriPartido        0
status.lider.uf                1
status.lider.idLegislatura     0
status.lider.urlFoto           0
data                           1
idLegislatura                  0
situacao                       1
totalPosse                     0
totalMembros                   0
uriMembros                     0
lider.uri 

In [20]:
# Renomeando colunas

df_partidos_completo = df_partidos_completo.rename(columns={'id': 'idPartido', 'sigla_x': 'siglaPartido', 'nome_x': 'nomePartido'})

In [21]:
# Excluindo colunas desnecessárias

df_partidos_completo.drop(columns=['uri_x', 'uri_y', 'urlLogo', 'urlWebSite', 'urlFacebook', 'status.lider.uri', 
                                   'status.lider.uriPartido', 'status.lider.urlFoto', 'uriMembros', 'lider.uri', 
                                   'lider.uriPartido', 'lider.urlFoto', 'uriMembros', 'lider.uri', 'lider.uriPartido', 
                                   'lider.urlFoto', 'sigla_y', 'nome_y', 'numeroEleitoral'], inplace=True)

In [22]:
# Formatando os campos de data para o formato brasileiro

df_partidos_completo['status.data'] = pd.to_datetime(df_partidos_completo['status.data']).dt.strftime("%d/%m/%Y")
df_partidos_completo['data'] = pd.to_datetime(df_partidos_completo['data']).dt.strftime("%d/%m/%Y")

In [23]:
# Verifica se pelo menos uma célula em cada linha (axis=1) tem o valor True, indicando que há um valor nulo nesta linha

df_partidos_completo[df_partidos_completo.isnull().any(axis=1)]

,idPartido,siglaPartido,nomePartido,status.data,status.idLegislatura,status.situacao,status.totalPosse,status.totalMembros,status.uriMembros,status.lider.nome,...,status.lider.idLegislatura,data,idLegislatura,situacao,totalPosse,totalMembros,lider.nome,lider.siglaPartido,lider.uf,lider.idLegislatura
25,36852,S.PART.,Sem Partido,NaN,57,None,0,0,,None,...,57,NaN,57,None,0,0,None,None,None,57


In [24]:
# Verificando quais colunas possuem campos nulos e filtrando para mostrar apenas as colunas que possuem esses valores nulos

df_partidos_completo.isnull().sum()[df_partidos_completo.isnull().sum() > 0]

status.data                  1
status.situacao              1
status.lider.nome            1
status.lider.siglaPartido    1
status.lider.uf              1
data                         1
situacao                     1
lider.nome                   1
lider.siglaPartido           1
lider.uf                     1
dtype: int64

In [25]:
# Preenchendo os campos nulos com o valor "Não se aplica" ou com a sigla do partido

df_partidos_completo.fillna({
    'status.data': 'Não se aplica',
    'status.situacao': 'Não se aplica',
    'status.lider.nome': 'Não se aplica',
    'status.lider.siglaPartido': df_partidos_completo['siglaPartido'],
    'status.lider.uf': 'Não se aplica',
    'data': 'Não se aplica',
    'situacao': 'Não se aplica',
    'lider.nome': 'Não se aplica',
    'lider.siglaPartido': df_partidos_completo['siglaPartido'],
    'lider.uf': 'Não se aplica' 
}, inplace=True)

In [26]:
# Criando o campo Está Ativo apenas com os partidos ativos, mas sem filtrar

df_partidos_completo['esta_ativo'] = df_partidos_completo['status.situacao'] == 'Ativo'

In [27]:
# Conferência final

df_partidos_completo.isnull().sum()

idPartido                     0
siglaPartido                  0
nomePartido                   0
status.data                   0
status.idLegislatura          0
status.situacao               0
status.totalPosse             0
status.totalMembros           0
status.uriMembros             0
status.lider.nome             0
status.lider.siglaPartido     0
status.lider.uf               0
status.lider.idLegislatura    0
data                          0
idLegislatura                 0
situacao                      0
totalPosse                    0
totalMembros                  0
lider.nome                    0
lider.siglaPartido            0
lider.uf                      0
lider.idLegislatura           0
esta_ativo                    0
dtype: int64

## 4.3. Tabela Líderes

In [28]:
# Verifica os campos da tabela em nulo

df_lideres_leg.isnull().sum()

titulo                       0
dataInicio                   0
dataFim                      8
parlamentar.id               0
parlamentar.uri              0
parlamentar.nome             0
parlamentar.siglaPartido     0
parlamentar.uriPartido       0
parlamentar.siglaUf          0
parlamentar.idLegislatura    0
parlamentar.email            8
parlamentar.urlFoto          0
bancada.tipo                 0
bancada.nome                 0
bancada.uri                  1
legislatura                  0
dtype: int64

In [29]:
# Filtra os campos dataFim que estão nulos e substitui pelas datas finais das legislaturas

filtro_nulo = df_lideres_leg["dataFim"].isnull()
df_lideres_leg.loc[filtro_nulo, "dataFim"] = df_lideres_leg.loc[filtro_nulo, "legislatura"].map({
    57: "2027-01-31",
    56: "2023-01-31"
})

In [30]:
# Transforma os campos de data para o padrão brasileiro

df_lideres_leg["dataInicio"] = pd.to_datetime(df_lideres_leg["dataInicio"]).dt.strftime("%d/%m/%Y")
df_lideres_leg["dataFim"] = pd.to_datetime(df_lideres_leg["dataFim"]).dt.strftime("%d/%m/%Y")

In [31]:
# Obtém os IDs únicos, removendo valores nulos
ids_unicos = df_lideres_leg['parlamentar.id'].dropna().unique()

# Obtém o DataFrame com os sexos baseados nos IDs únicos
df_sexo_por_id = obter_sexo_por_id(ids_unicos)

# Realiza o merge
df_lideres_leg = df_lideres_leg.merge(df_sexo_por_id, left_on='parlamentar.id', right_on='id', how='left')

In [32]:
# Contagem de True/False para a comparação 'parlamentar.id' com 'id'

count_id_true = (df_lideres_leg['parlamentar.id'] == df_lideres_leg['id']).sum()
count_id_false = len(df_lideres_leg) - count_id_true  # Total de registros - True -> False
print(f"Quantidade de True para 'parlamentar.id' e 'id': {count_id_true}")
print(f"Quantidade de False para 'parlamentar.id' e 'id': {count_id_false}")

# Contagem de True/False para a comparação 'parlamentar.idLegislatura' com 'legislatura'

count_legislatura_true = (df_lideres_leg['parlamentar.idLegislatura'] == df_lideres_leg['legislatura']).sum()
count_legislatura_false = len(df_lideres_leg) - count_legislatura_true
print(f"Quantidade de True para 'parlamentar.idLegislatura' e 'legislatura': {count_legislatura_true}")
print(f"Quantidade de False para 'parlamentar.idLegislatura' e 'legislatura': {count_legislatura_false}")

Quantidade de True para 'parlamentar.id' e 'id': 30
Quantidade de False para 'parlamentar.id' e 'id': 0
Quantidade de True para 'parlamentar.idLegislatura' e 'legislatura': 30
Quantidade de False para 'parlamentar.idLegislatura' e 'legislatura': 0


In [33]:
# Exclui colunas desnecessárias

df_lideres_leg.drop(columns=['parlamentar.email', 'bancada.uri', 'parlamentar.idLegislatura', 'id'], inplace=True)

In [34]:
# Conferência final

df_lideres_leg.isnull().sum()

titulo                      0
dataInicio                  0
dataFim                     0
parlamentar.id              0
parlamentar.uri             0
parlamentar.nome            0
parlamentar.siglaPartido    0
parlamentar.uriPartido      0
parlamentar.siglaUf         0
parlamentar.urlFoto         0
bancada.tipo                0
bancada.nome                0
legislatura                 0
sexo                        0
dtype: int64

## 4.4. Tabela Proposições

In [35]:
# ====================== Função para Requisição com Tentativas ======================
def fazer_requisicao_com_tentativas(url, tentativas=3, delay=5):
    session = requests.Session()
    for tentativa in range(tentativas):
        try:
            response = session.get(url)
            response.raise_for_status()
            return response.json()
        except requests.RequestException as e:
            print(f"Tentativa {tentativa + 1} falhou. Erro: {e}")
            if tentativa < tentativas - 1:
                time.sleep(delay)
            else:
                print(f"Erro permanente na URL: {url}")
                return None

# ====================== Função para Coletar Proposições ======================
def coletar_proposicoes(df_deputadas_completo, pasta_dados='dados_proposicoes'):
    # Cria a pasta, se não existir
    os.makedirs(pasta_dados, exist_ok=True)
    inicio_tempo = time.time()
    
    # Itera sobre cada ano do período
    for ano in range(2019, 2025):  # De 2019 a 2024
        arquivo_ano = f"{pasta_dados}/proposicoes_{ano}.csv"
        
        # Verifica se o arquivo do ano já existe
        if os.path.exists(arquivo_ano):
            print(f"[INFO] Dados para {ano} já existem. Pulando...")
            continue
        
        todos_dados = []  # Lista para armazenar todas as proposições do ano
        
        # Itera sobre cada deputada
        for id_deputada in df_deputadas_completo['id'].unique():
            pagina_atual = 1
            url_base = f"https://dadosabertos.camara.leg.br/api/v2/proposicoes?idDeputadoAutor={id_deputada}&dataApresentacaoInicio={ano}-01-01&dataApresentacaoFim={ano}-12-31&ordem=ASC&ordenarPor=id&itens=100"
            url_proxima = url_base
            
            while url_proxima:
                dados_pagina = fazer_requisicao_com_tentativas(url_proxima)
                
                if dados_pagina:
                    # Adiciona os dados da página atual, incluindo o ID do deputado
                    for dado in dados_pagina.get('dados', []):
                        dado['idDeputadoAutor'] = id_deputada
                        todos_dados.append(dado)
                    
                    # Verifica a URL da próxima página
                    url_proxima = next((link['href'] for link in dados_pagina.get('links', []) if link['rel'] == 'next'), None)
                    
                    # Imprime progresso
                    print(f"Processando ID {id_deputada} - Ano {ano} - Página {pagina_atual}")
                    pagina_atual += 1
                else:
                    break  # Encerra o loop em caso de erro na requisição
        
        # Cria o DataFrame final do ano e salva em CSV
        if todos_dados:
            df_proposicoes_ano = pd.DataFrame(todos_dados)
            df_proposicoes_ano.to_csv(arquivo_ano, index=False, encoding="utf-8")
            print(f"[SUCESSO] Dados para {ano} salvos em {arquivo_ano}")
    
    # Exibe tempo total de execução
    tempo_total = time.time() - inicio_tempo
    print(f"Tempo total de execução: {tempo_total:.2f} segundos")
    print("[FINALIZADO] Todos os anos processados com sucesso.")

In [36]:
# Uso da Função Coletar proposições (executar apenas se necessário)
# df_proposicoes = coletar_proposicoes(df_deputadas_completo)

In [37]:
# ====================== Carregamento dos Dados Coletados ======================
def carregar_proposicoes(pasta_dados='../data/dados_proposicoes'):    # Caminho para a pasta com os CSVs
    todos_anos = []                                           # Lista para armazenar os DataFrames
    for arquivo in os.listdir(pasta_dados):                   # Itera sobre os arquivos na pasta
        if arquivo.endswith('.csv'):
            ano = arquivo.split('_')[-1].split('.')[0]        # Extrai o ano do nome do arquivo
            df = pd.read_csv(os.path.join(pasta_dados, arquivo))
            df['ano'] = ano                                   # Adiciona a coluna de ano para identificar a origem
            todos_anos.append(df)
    return pd.concat(todos_anos, ignore_index=True) if todos_anos else pd.DataFrame()  # Concatena todos os DataFrames em um único DataFrame

In [38]:
# Uso da Função Carregar proposições (executar apenas se necessário)
# df_proposicoes_completo = carregar_proposicoes()
# print(f"Total de proposições carregadas: {len(df_proposicoes_completo)}")

In [39]:
# Arquivo baixado em 13/05/2025 e utilizado para análise
df_proposicoes_completo = pd.read_csv('../data/proposicoes_com_sexo.csv', sep=';')

In [40]:
# ====================== Extração dos Detalhes ======================
def extrair_detalhes(df_proposicoes_completo):
    detalhes = []
    qtde_ids_unicos = 0              # Inicializa o contador

    print("Início do processamento...")
    for id in df_proposicoes_completo['id'].unique():
        try:
            url = f'https://dadosabertos.camara.leg.br/api/v2/proposicoes/{id}'
            data = fazer_requisicao_com_tentativas(url)

            # Verifica se a resposta não está vazia
            if data and "dados" in data:
                df_dados = pd.json_normalize(data["dados"])
                detalhes.append(df_dados)
            
            qtde_ids_unicos += 1
            print(f'Proposições processadas: {qtde_ids_unicos} / {len(df_proposicoes_completo["id"].unique())}')
        
        except Exception as e:
            print(f"[ERRO] Falha ao processar a proposição {id}: {e}")

    df_proposicoes_detalhes = pd.concat(detalhes, ignore_index=True) if detalhes else pd.DataFrame()
    print("Processamento finalizado!")
    print(df_proposicoes_detalhes.info())
    return df_proposicoes_detalhes

In [41]:
# df_proposicoes_detalhes = extrair_detalhes(df_proposicoes_completo)

In [42]:
# df_proposicoes_detalhes.to_csv('proposicoes_detalhes.csv', index=False)

In [43]:
# Arquivo baixado em 29/05/2025 e utilizado para análise
df_proposicoes_detalhes = pd.read_csv('../data/proposicoes_detalhes.csv', sep=',')

In [44]:
df_proposicoes_detalhes['id'].count()

135068

In [45]:
# ====================== Extração dos Temas ======================
def extrair_detalhes(df_proposicoes_completo):
    detalhes = []
    qtde_ids_unicos = 0              # Inicializa o contador

    print("Início do processamento...")
    for id in df_proposicoes_completo['id'].unique():
        try:
            url = f'https://dadosabertos.camara.leg.br/api/v2/proposicoes/{id}/temas'
            data = fazer_requisicao_com_tentativas(url)

            # Verifica se a resposta não está vazia
            if data and "dados" in data:
                df_dados = pd.json_normalize(data["dados"])
                df_dados['idProposicao'] = id              # Adiciona o ID da proposição ao DataFrame
                detalhes.append(df_dados)
            
            qtde_ids_unicos += 1
            print(f'Proposições processadas: {qtde_ids_unicos} / {len(df_proposicoes_completo["id"].unique())}')
        
        except Exception as e:
            print(f"[ERRO] Falha ao processar a proposição {id}: {e}")

    df_proposicoes_tema = pd.concat(detalhes, ignore_index=True) if detalhes else pd.DataFrame()
    print("Processamento finalizado!")
    print(df_proposicoes_tema.info())
    return df_proposicoes_tema

In [46]:
# Extrair temas das proposições (executar apenas se necessário)
# df_proposicoes_tema = extrair_detalhes(df_proposicoes_completo)

In [47]:
# Arquivo baixado em 13/05/2025 e utilizado para análise
df_proposicoes_tema = pd.read_csv('../data/proposicoes_temas.csv', sep=',')

In [48]:
df_proposicoes_completo.isnull().sum()

idProposicao       0
uri                0
siglaTipo          0
codTipo            0
numero             0
ano                0
idDeputadoAutor    0
sexo               0
dtype: int64

In [49]:
df_proposicoes_completo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 318208 entries, 0 to 318207
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   idProposicao     318208 non-null  int64 
 1   uri              318208 non-null  object
 2   siglaTipo        318208 non-null  object
 3   codTipo          318208 non-null  int64 
 4   numero           318208 non-null  int64 
 5   ano              318208 non-null  int64 
 6   idDeputadoAutor  318208 non-null  int64 
 7   sexo             318208 non-null  object
dtypes: int64(5), object(3)
memory usage: 19.4+ MB


In [50]:
df_proposicoes_completo.dropna(axis=0, how='any', inplace=True)

In [51]:
df_proposicoes_completo['numero'] = df_proposicoes_completo['numero'].round().astype('int64')

In [52]:
# df_proposicoes_completo.drop(columns=['ementa'], inplace=True)

In [53]:
df_proposicoes_completo.isnull().sum()

idProposicao       0
uri                0
siglaTipo          0
codTipo            0
numero             0
ano                0
idDeputadoAutor    0
sexo               0
dtype: int64

In [54]:
# df_proposicoes_completo.to_csv('proposicoes.csv', sep=';', index=False)

In [55]:
# Fazendo o merge com base nos IDs
df_proposicoes_com_sexo = df_proposicoes_completo.merge(
    df_deputadas_completo[['id', 'sexo']],  # Filtra apenas as colunas necessárias
    left_on='idDeputadoAutor', 
    right_on='id', 
    how='left'  # left join para manter todas as proposições
)

# Exibindo algumas linhas para verificar se funcionou
print(df_proposicoes_com_sexo.head())

   idProposicao                                                uri siglaTipo  \
0       2199501  https://dadosabertos.camara.leg.br/api/v2/prop...        PL   
1       2199501  https://dadosabertos.camara.leg.br/api/v2/prop...        PL   
2       2199624  https://dadosabertos.camara.leg.br/api/v2/prop...       EMC   
3       2199624  https://dadosabertos.camara.leg.br/api/v2/prop...       EMC   
4       2199625  https://dadosabertos.camara.leg.br/api/v2/prop...       EMC   

   codTipo  numero   ano  idDeputadoAutor sexo_x      id sexo_y  
0      139    2552  2019           204554      M  204554      M  
1      139    2552  2019           204554      M  204554      M  
2      130       1  2019           204554      M  204554      M  
3      130       1  2019           204554      M  204554      M  
4      130       2  2019           204554      M  204554      M  


In [56]:
# Removendo a coluna id duplicada
df_proposicoes_com_sexo.drop(columns=['id', 'sexo_y'], inplace=True)
df_proposicoes_com_sexo.rename(columns={"sexo_x": "sexo"}, inplace=True)

In [57]:
print(df_proposicoes_com_sexo.columns)

Index(['idProposicao', 'uri', 'siglaTipo', 'codTipo', 'numero', 'ano',
       'idDeputadoAutor', 'sexo'],
      dtype='object')


In [58]:
df_proposicoes_com_sexo['idProposicao'].count()

633307

In [59]:
print("Registros antes do merge:", len(df_proposicoes_completo))
print("Registros após o merge:", len(df_proposicoes_com_sexo))

Registros antes do merge: 318208
Registros após o merge: 633307


In [60]:
print(df_proposicoes_com_sexo['sexo'].isnull().sum())

0


In [61]:
deputados_nao_encontrados = df_proposicoes_com_sexo[df_proposicoes_com_sexo['sexo'].isnull()]['idDeputadoAutor'].unique()
print("Deputados não encontrados:", deputados_nao_encontrados)

Deputados não encontrados: []


In [62]:
# Verificando duplicatas na tabela de deputadas
duplicatas_deputadas = df_deputadas_completo[df_deputadas_completo.duplicated(subset=['id', 'siglaPartido', 'idLegislatura'], keep=False)]
print("Deputados com IDs duplicados:\n", duplicatas_deputadas[['id', 'nome', 'siglaPartido', 'idLegislatura']].sort_values(by='id'))
print("Total de duplicatas:", len(duplicatas_deputadas))

Deputados com IDs duplicados:
 Empty DataFrame
Columns: [id, nome, siglaPartido, idLegislatura]
Index: []
Total de duplicatas: 0


In [63]:
# Contando quantas proposições cada deputado fez

contagem_proposicoes = df_proposicoes_completo['idDeputadoAutor'].value_counts()
print(contagem_proposicoes.describe())

count     547.000000
mean      581.733090
std       588.553173
min         1.000000
25%       232.000000
50%       435.000000
75%       716.000000
max      4680.000000
Name: count, dtype: float64


In [64]:
# Excluindo os ids que foram duplicados após o merge

df_proposicoes_com_sexo.drop_duplicates(keep='last', inplace=True)
print("Registros antes do merge:", len(df_proposicoes_completo))
print("Registros após o merge:", len(df_proposicoes_com_sexo))

Registros antes do merge: 318208
Registros após o merge: 318208


In [65]:
# df_proposicoes_com_sexo.to_csv('proposicoes_com_sexo.csv', sep=';', index=False)

In [66]:
df_proposicoes_tema.isnull().sum()

codTema         0
tema            0
relevancia      0
idProposicao    0
dtype: int64

In [67]:
df_proposicoes_tema.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63161 entries, 0 to 63160
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   codTema       63161 non-null  float64
 1   tema          63161 non-null  object 
 2   relevancia    63161 non-null  float64
 3   idProposicao  63161 non-null  int64  
dtypes: float64(2), int64(1), object(1)
memory usage: 1.9+ MB


In [68]:
df_proposicoes_tema['codTema'] = df_proposicoes_tema['codTema'].round().astype('int64')

geral: https://dadosabertos.camara.leg.br/api/v2/proposicoes/2199501
autores: https://dadosabertos.camara.leg.br/api/v2/proposicoes?idDeputadoAutor=204554&dataApresentacaoInicio=2019-01-01&dataApresentacaoFim=2024-12-31&ordem=ASC&ordenarPor=id&itens=50


temas: https://dadosabertos.camara.leg.br/api/v2/proposicoes/2199501/temas
tramitacoes: https://dadosabertos.camara.leg.br/api/v2/proposicoes/2199501/tramitacoes
votacoes: https://dadosabertos.camara.leg.br/api/v2/proposicoes/2199501/votacoes?ordem=DESC&ordenarPor=dataHoraRegistro



url_cod_situacao_proposicao = 'https://dadosabertos.camara.leg.br/api/v2/referencias/proposicoes/codSituacao'
url_cod_tema_proposicao = 'https://dadosabertos.camara.leg.br/api/v2/referencias/proposicoes/codTema'
url_sigla_tipo_proposicao = 'https://dadosabertos.camara.leg.br/api/v2/referencias/proposicoes/siglaTipo'
url_cod_tramitacao_proposicao = 'https://dadosabertos.camara.leg.br/api/v2/referencias/proposicoes/codTipoTramitacao'
url_cod_tipo_autor_proposicao = 'https://dadosabertos.camara.leg.br/api/v2/referencias/proposicoes/codTipoAutor'

# 5. Análise Exploratória

In [69]:
df_proposicoes_completo.groupby('ano')['idDeputadoAutor'].nunique()

ano
2019    346
2020    340
2021    345
2022    347
2023    365
2024    369
Name: idDeputadoAutor, dtype: int64

In [70]:
df_proposicoes_completo.groupby('ano')['idProposicao'].nunique()

ano
2019    21100
2020    14693
2021    21273
2022    13172
2023    33987
2024    30843
Name: idProposicao, dtype: int64

In [71]:
df_proposicoes_completo['idProposicao'].nunique()

135068

In [72]:
df_proposicoes_tema['idProposicao'].count()

63161

In [73]:
df_proposicoes_tema['idProposicao'].nunique()

33856

In [74]:
df_proposicoes_tema.groupby('tema')['idProposicao'].nunique()

tema
Administração Pública                          6910
Agricultura, Pecuária, Pesca e Extrativismo    1154
Arte, Cultura e Religião                        896
Cidades e Desenvolvimento Urbano               1382
Ciência, Tecnologia e Inovação                  690
Ciências Exatas e da Terra                        2
Ciências Sociais e Humanas                        7
Comunicações                                   1134
Defesa e Segurança                             3439
Direito Civil e Processual Civil               1203
Direito Constitucional                          157
Direito Penal e Processual Penal               2465
Direito e Defesa do Consumidor                 1314
Direito e Justiça                               433
Direitos Humanos e Minorias                    6228
Economia                                       2053
Educação                                       3562
Energia, Recursos Hídricos e Minerais          1658
Esporte e Lazer                                 632
Estrutu

In [75]:
df_proposicoes_com_sexo.groupby('siglaTipo', dropna=False).size()

siglaTipo
ADD         1
APM         6
ATA         2
ATACN     195
CAE        14
         ... 
SLD       658
SOR      1224
SPP        35
SSP         4
VTS       593
Length: 80, dtype: int64

In [76]:
df_proposicoes_com_sexo.groupby('sexo', dropna=False)['idProposicao'].nunique()

sexo
F     33978
M    110739
Name: idProposicao, dtype: int64

In [77]:
tabela_contagem = df_proposicoes_com_sexo.pivot_table(index='ano', columns='sexo', values='idProposicao', aggfunc='nunique')
print(tabela_contagem)

sexo     F      M
ano              
2019  4230  17776
2020  3701  12686
2021  5070  17671
2022  2690  11167
2023  8901  27332
2024  9386  24107


In [78]:
# Cria a tabela de contagem absoluta de idProposicao únicos
tabela_absoluta = df_proposicoes_com_sexo.pivot_table(
    index='ano', 
    columns='sexo', 
    values='idProposicao', 
    aggfunc='nunique', 
    fill_value=0
)

# Calcula os percentuais
tabela_percentual = (tabela_absoluta.div(tabela_absoluta.sum(axis=1), axis=0) * 100).round(2)

# Combina os dois DataFrames (absoluto e percentual)
tabela_completa = tabela_absoluta.astype(str) + " (" + tabela_percentual.astype(str) + "%)"

# Exibe a tabela formatada
tabela_completa.style.set_caption("Proposições Únicas por Ano e Gênero (Absoluto e %)").set_properties(**{'text-align': 'center'})

sexo,F,M
ano,,
2019,4230 (19.22%),17776 (80.78%)
2020,3701 (22.58%),12686 (77.42%)
2021,5070 (22.29%),17671 (77.71%)
2022,2690 (19.41%),11167 (80.59%)
2023,8901 (24.57%),27332 (75.43%)
2024,9386 (28.02%),24107 (71.98%)


In [79]:
print(df_lideres_leg[['parlamentar.siglaPartido', 'legislatura', 'parlamentar.nome', 'sexo']])

   parlamentar.siglaPartido  legislatura        parlamentar.nome sexo
0                     UNIÃO           56             José Rocha     M
1                       PSB           56          Tadeu Alencar     M
2                     PCdoB           56          Orlando Silva     M
3                       PSC           56    Gilberto Nascimento     M
4                        PT           56          Paulo Pimenta     M
5                     UNIÃO           56  Pedro Lucas Fernandes     M
6                       PSD           56         André de Paula     M
7              REPUBLICANOS           56      Jhonatan de Jesus     M
8                      PSDB           56         Carlos Sampaio     M
9                 CIDADANIA           56          Daniel Coelho     M
10                       PP           56             José Nelto     M
11                      MDB           56           Baleia Rossi     M
12                   AVANTE           56              Luis Tibé     M
13                  

In [80]:
df_lideres_leg.groupby('sexo', dropna=False).size()

sexo
F     3
M    27
dtype: int64

In [81]:
df_lideres_leg.groupby('parlamentar.siglaPartido')['legislatura'].count()

parlamentar.siglaPartido
AVANTE          3
CIDADANIA       1
MDB             3
PCdoB           1
PDT             1
PODE            2
PP              2
PRD             1
PSB             2
PSC             1
PSD             2
PSDB            2
PT              3
REDE            1
REPUBLICANOS    3
UNIÃO           2
Name: legislatura, dtype: int64

In [82]:
# Verificando se a tabela de Deputadas possui apenas duas legislaturas

df_deputadas_completo.groupby('idLegislatura')['idLegislatura'].nunique(dropna=False)

idLegislatura
56    1
57    1
Name: idLegislatura, dtype: int64

In [83]:
# Verificando as siglas dos Partidos que são únicas na tabela de Deputadas

df_deputadas_completo['siglaPartido'].nunique()

31

In [84]:
# Agrupando os registros pelo campo sexo, com base na contagem única de ids da tabela de Deputadas

df_deputadas_completo.groupby('sexo', dropna=False)['id'].nunique(dropna=False)

sexo
F    103
M    449
Name: id, dtype: int64

In [85]:
df_deputadas_completo.groupby('idLegislatura')['id'].nunique()

idLegislatura
56    378
57    384
Name: id, dtype: int64

In [86]:
# Cria uma tabela de id_deputado com os anos em que apareceram
tabela_legislaturas = df_deputadas_completo.groupby('id')['idLegislatura'].nunique().reset_index()

# Deputadas que aparecem em dois anos => reeleitas
tabela_legislaturas['reeleita'] = tabela_legislaturas['idLegislatura'] > 1


In [87]:
# Junta com o DataFrame original
df_deputadas_completo = df_deputadas_completo.merge(tabela_legislaturas[['id', 'reeleita']], on='id', how='left')


In [88]:
deputadas_reeleitas = df_deputadas_completo[df_deputadas_completo['reeleita']].drop_duplicates('id')
print(f"Deputadas reeleitas: {len(deputadas_reeleitas)}")


Deputadas reeleitas: 210


In [89]:
novatas_2023 = df_deputadas_completo[(df_deputadas_completo['idLegislatura'] == 57) & (~df_deputadas_completo['reeleita'])]
novatas_2023 = novatas_2023.drop_duplicates('id')
print(f"Deputadas novatas em 2023: {len(novatas_2023)}")

Deputadas novatas em 2023: 174


In [90]:
total_2019 = df_deputadas_completo[df_deputadas_completo['idLegislatura'] == 56]['id'].nunique()
reeleitas = deputadas_reeleitas['id'].nunique()
percentual_reeleicao = (reeleitas / total_2019) * 100
print(f"Percentual de reeleição entre deputadas de 2019: {percentual_reeleicao:.2f}%")


Percentual de reeleição entre deputadas de 2019: 55.56%


In [91]:
def contar_deputadas(df, legislatura=None, tipo='total'):
    """
    Conta deputadas eleitas, com opção de contagem total (mandatos) ou distinta (únicas).
    
    Parâmetros:
    - df: DataFrame com as deputadas
    - legislatura: int ou lista (ex: 56 ou [56, 57])
    - tipo: 'total' para contar mandatos, 'distinta' para contar deputadas únicas
    
    Retorna: inteiro com o número de deputadas
    """
    df_fem = df[df['sexo'] == 'F']
    
    if legislatura:
        if isinstance(legislatura, int):
            df_fem = df_fem[df_fem['idLegislatura'] == legislatura]
        elif isinstance(legislatura, list):
            df_fem = df_fem[df_fem['idLegislatura'].isin(legislatura)]

    if tipo == 'total':
        return len(df_fem)
    elif tipo == 'distinta':
        return df_fem['id'].nunique()
    else:
        raise ValueError("Tipo inválido: use 'total' ou 'distinta'")


In [92]:
print("🔍 Exemplos com df_deputadas_completo\n")

# Total de mandatos femininos em 2019 (legislatura 56)
total_2019 = contar_deputadas(df_deputadas_completo, 56, tipo='total')
print(f"Total de mandatos femininos em 2019: {total_2019}")

# Total de mandatos femininos em 2023 (legislatura 57)
total_2023 = contar_deputadas(df_deputadas_completo, 57, tipo='total')
print(f"Total de mandatos femininos em 2023: {total_2023}")

# Total de mandatos (mesmo nome conta duas vezes) entre 2019 e 2023
total_ambos = contar_deputadas(df_deputadas_completo, [56, 57], tipo='total')
print(f"Total de mandatos femininos entre 2019 e 2023: {total_ambos}")

# Total de deputadas únicas em 2019
distintas_2019 = contar_deputadas(df_deputadas_completo, 56, tipo='distinta')
print(f"Total de deputadas únicas em 2019: {distintas_2019}")

# Total de deputadas únicas em 2023
distintas_2023 = contar_deputadas(df_deputadas_completo, 57, tipo='distinta')
print(f"Total de deputadas únicas em 2023: {distintas_2023}")

# Total de deputadas únicas entre 2019 e 2023
distintas_ambos = contar_deputadas(df_deputadas_completo, [56, 57], tipo='distinta')
print(f"Total de deputadas únicas entre 2019 e 2023: {distintas_ambos}")

🔍 Exemplos com df_deputadas_completo

Total de mandatos femininos em 2019: 91
Total de mandatos femininos em 2023: 76
Total de mandatos femininos entre 2019 e 2023: 167
Total de deputadas únicas em 2019: 56
Total de deputadas únicas em 2023: 74
Total de deputadas únicas entre 2019 e 2023: 103


In [93]:
def gerar_calendario_legislaturas(df):
    """Gera calendário de datas únicas por legislatura"""
    start_time = time.time()
    print("Baixando calendário...")

    # Lista de DataFrames: um para cada legislatura
    df_calendario_list = [
        pd.DataFrame({
            'data': pd.date_range(start=row['inicioLegislatura'], end=row['fimLegislatura']),
            'idLegislatura': row['idLegislatura']
        })
        for _, row in df.iterrows()
    ]

    # Concatenar e remover duplicatas no par (data, idLegislatura)
    df_calendario = (
        pd.concat(df_calendario_list, ignore_index=True)
          .drop_duplicates(subset=['data', 'idLegislatura'])
    )

    # Formatar a coluna data
    df_calendario['data'] = df_calendario['data'].dt.strftime('%d/%m/%Y')

    elapsed = time.time() - start_time
    print(f"Tempo total de execução: {elapsed:.2f} segundos")

    return df_calendario

In [94]:
# Chamada da função e download da tabela
# df_calendario = gerar_calendario_legislaturas(df_deputadas_completo)
# df_calendario.to_csv("calendario.csv", index=False)

In [95]:
# Etapa 1: Unir proposições com os detalhes para obter a data de apresentação
# a. Renomear colunas para snake_case
df_proposicoes_completo = df_proposicoes_completo.rename(columns={
    'idProposicao': 'id_proposicao',
    'idDeputadoAutor' : 'id_deputado_autor'
})

df_proposicoes_detalhes = df_proposicoes_detalhes.rename(columns={
    'id': 'id_proposicao',
    'dataApresentacao': 'data_apresentacao'
})

# b. Fazer o merge usando as colunas já em snake_case
df_proposicoes_full = df_proposicoes_completo.merge(
    df_proposicoes_detalhes[['id_proposicao', 'data_apresentacao']],
    on='id_proposicao',
    how='inner'
)

# Etapa 2: Preparar tabela de legislaturas com início e fim
df_legislaturas = df_deputadas_completo[[
    'id', 'idLegislatura', 'inicioLegislatura', 'fimLegislatura'
]].copy()

df_legislaturas.rename(columns={
    'id': 'id_deputada',
    'idLegislatura': 'id_legislatura',
    'inicioLegislatura': 'inicio',
    'fimLegislatura': 'fim'
}, inplace=True)

# Garantir que datas estão em formato datetime
df_legislaturas['inicio'] = pd.to_datetime(df_legislaturas['inicio'], dayfirst=True, errors='coerce')
df_legislaturas['fim'] = pd.to_datetime(df_legislaturas['fim'], dayfirst=True, errors='coerce')
df_proposicoes_full['data_apresentacao'] = pd.to_datetime(df_proposicoes_full['data_apresentacao'], errors='coerce')

# Etapa 3: Merge pelo autor com faixa de datas
df_merged = df_proposicoes_full.merge(
    df_legislaturas,
    left_on='id_deputado_autor',
    right_on='id_deputada',
    how='left'
)

# Etapa 4: Filtrar registros onde a proposição foi apresentada dentro do mandato
df_validos = df_merged[
    (df_merged['data_apresentacao'] >= df_merged['inicio']) &
    (df_merged['data_apresentacao'] <= df_merged['fim'])
].copy()

# Etapa 5: Criar a chave composta id_deputada_legislatura
df_validos['id_deputada_legislatura'] = (
    df_validos['id_deputado_autor'].astype(str) + "-" + df_validos['id_legislatura'].astype(str)
)

# Etapa 6: Gerar resultado final
df_proposicoes_autores = df_validos[['id_proposicao', 'id_deputada_legislatura']].drop_duplicates()

# Visualizar resultado
df_proposicoes_autores.head()

,id_proposicao,id_deputada_legislatura
0,2199501,204554-56
2,2199624,204554-56
4,2199625,204554-56
6,2199779,204554-56
8,2200893,204554-56


In [96]:
df_lideres_leg.columns

Index(['titulo', 'dataInicio', 'dataFim', 'parlamentar.id', 'parlamentar.uri',
       'parlamentar.nome', 'parlamentar.siglaPartido',
       'parlamentar.uriPartido', 'parlamentar.siglaUf', 'parlamentar.urlFoto',
       'bancada.tipo', 'bancada.nome', 'legislatura', 'sexo'],
      dtype='object')

In [97]:
df_lideres_leg = df_lideres_leg.rename(columns={'parlamentar.id': 'id'})
df_lideres = obter_detalhes_deputadas(df_lideres_leg)

Tempo total de execução: 4.16 segundos


In [98]:
df_lideres.isnull().sum()

id                                 0
uri                                0
nomeCivil                          0
cpf                                0
sexo                               0
urlWebsite                        30
redeSocial                         0
dataNascimento                     0
dataFalecimento                   30
ufNascimento                       0
municipioNascimento                0
escolaridade                       0
ultimoStatus.id                    0
ultimoStatus.uri                   0
ultimoStatus.nome                  0
ultimoStatus.siglaPartido          0
ultimoStatus.uriPartido           30
ultimoStatus.siglaUf               0
ultimoStatus.idLegislatura         0
ultimoStatus.urlFoto               0
ultimoStatus.email                30
ultimoStatus.data                  0
ultimoStatus.nomeEleitoral         0
ultimoStatus.gabinete.nome         8
ultimoStatus.gabinete.predio       8
ultimoStatus.gabinete.sala         8
ultimoStatus.gabinete.andar        9
u

In [99]:
df_lideres['falecida'] = df_lideres['dataFalecimento'].notna()
df_lideres.loc[:, 'ano_nascimento'] = pd.to_datetime(df_lideres['dataNascimento'], dayfirst=True).dt.year
df_lideres.loc[:, 'idade'] = datetime.now().year - df_lideres['ano_nascimento']
df_lideres.loc[:, 'geração'] = df_lideres['ano_nascimento'].apply(determinar_geracao)
df_lideres.drop(columns=['uri', 'cpf', 'urlWebsite', 'redeSocial', 'ultimoStatus.uriPartido', 'dataFalecimento', 'ultimoStatus.idLegislatura',
                                    'ultimoStatus.id', 'ultimoStatus.uri', 'ultimoStatus.gabinete.nome',
                                    'ultimoStatus.urlFoto', 'ultimoStatus.email', 'ultimoStatus.data', 'ultimoStatus.nomeEleitoral',
                                    'ultimoStatus.gabinete.predio', 'ultimoStatus.gabinete.sala', 'ultimoStatus.gabinete.andar',
                                    'ultimoStatus.gabinete.telefone', 'ultimoStatus.gabinete.email', 'ultimoStatus.descricaoStatus', 
                                    'ultimoStatus.gabinete.nome', 'ultimoStatus.situacao', 'ultimoStatus.condicaoEleitoral'], inplace=True)

C:\Users\caren\AppData\Local\Temp\ipykernel_24836\531666320.py:2: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df_lideres.loc[:, 'ano_nascimento'] = pd.to_datetime(df_lideres['dataNascimento'], dayfirst=True).dt.year


In [100]:
df_lideres.isnull().sum()

id                           0
nomeCivil                    0
sexo                         0
dataNascimento               0
ufNascimento                 0
municipioNascimento          0
escolaridade                 0
ultimoStatus.nome            0
ultimoStatus.siglaPartido    0
ultimoStatus.siglaUf         0
falecida                     0
ano_nascimento               0
idade                        0
geração                      0
dtype: int64

In [102]:
# df_lideres.to_csv('dim_liderancas.csv', index=False)